In [3]:
%run -i "../classes.py"
%run -i "../functions.py"

import pickle
import pandas as pd

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package words to
[nltk_data]     C:\Users\adamm\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


In [ ]:
"../all_datasets/"

In [22]:
model_settings_all_datasets = {
     "best_word2vec":
    {
        "model_path": "../all_datasets/word2vec_models/word2vec_vs50_win5_sg0.model",
        "vector_size": 50,
        "window_size": 5,
        "is_skipgram": False
    }
}

model_settings_clickbait_dataset = {
     "best_word2vec":
    {
        "model_path": "../all_datasets/word2vec_models/word2vec_vs20_win4_sg1.model",
        "vector_size": 20,
        "window_size": 4,
        "is_skipgram": True
    }
}

model_settings_fake_news = {
     "best_word2vec":
    {
        "model_path": "../all_datasets/word2vec_models/word2vec_vs100_win3_sg0.model",
        "vector_size": 100,
        "window_size": 3,
        "is_skipgram": False
    }
}


model_w2v_all_datasets = Word2VecModel(model_settings_all_datasets["best_word2vec"])
model_w2v_clickbait_dataset = Word2VecModel(model_settings_clickbait_dataset["best_word2vec"])
model_w2v_fake_news = Word2VecModel(model_settings_fake_news["best_word2vec"])

In [25]:
variables_to_drop_all_datasets = get_dimensions_to_drop("../all_datasets/")
variables_to_drop_clickbait_dataset = get_dimensions_to_drop("../clickbait_dataset/")
variables_to_drop_fake_news = get_dimensions_to_drop("../fake_news_dataset/")

FileNotFoundError: [Errno 2] No such file or directory: '../clickbait_dataset/predictive_models/worst_performing_dimensions_intersection.pkl'

In [9]:
with open('../data/preprocessed_titles_labels.pkl', 'rb') as f:
    data = pickle.load(f)

df = pd.DataFrame(data)
df=df[df['sample']=='test'].reset_index(drop=True)

df['title_vector_all_datasets'] = [get_word_vectors(model_w2v_all_datasets, title, aggregation='mean') for title in df['title']]
df['title_vector_clickbait-dataset'] = [get_word_vectors(model_w2v_clickbait_dataset, title, aggregation='mean') for title in df['title']]
df['title_vector_fake-news'] = [get_word_vectors(model_w2v_fake_news, title, aggregation='mean') for title in df['title']]

df['title_vector_all_datasets'] = [drop_dimensions_from_vector(vector, variables_to_drop_all_datasets) for vector in df['title_vector_all_datasets']]
df['title_vector_clickbait-dataset'] = [drop_dimensions_from_vector(vector, variables_to_drop_clickbait_dataset) for vector in df['title_vector_clickbait-dataset']]
df['title_vector_fake-news'] = [drop_dimensions_from_vector(vector, variables_to_drop_fake_news) for vector in df['title_vector_fake-news']]

df.head()

,title,is_clickbait,text,dataset,sample
0,"[florida, back, away, prep, schedule, favoring...",0,NaN,clickbait-dataset,test
1,"[beck, comey, letter, , one, irresponsible, t...",1,Home › POLITICS › BECK: COMEY LETTER ‘ONE OF T...,fake-news,test
2,"[sydney, swelter, climate, change, link, scien...",0,Southeastern Australia has suffered through a ...,fake-news,test
3,"[california, today, airbnb, hometown, success,...",0,Good morning. This is the last day of our test...,fake-news,test
4,"[clinton, could, win, control, voting, machine...",1,Voter Fraud is Trump’s Greatest Challenge New ...,fake-news,test


In [11]:
import copy as cp
df_all_datasets = cp.deepcopy(df)
df_clickbait_dataset = df[df['dataset']=='clickbait-dataset'].reset_index(drop=True)
df_fake_news = df[df['dataset']=='fake-news'].reset_index(drop=True)

df_all_datasets.shape, df_clickbait_dataset.shape, df_fake_news.shape


((2590, 5), (1595, 5), (995, 5))

In [7]:
model_combinations = ['mod_all_datasets', 'mod_fake-news', 'mod_clickbait-dataset-small', 'mod_clickbait-dataset-large']
dataset_combinations = ['data_all_datasets', 'data_fake-news', 'data_clickbait-dataset']
metrics = ['accuracy', 'precision', 'recall', 'f1-score', 'roc-auc']
tresholds = ['default', 'optimizedF1']

results = dict()

for model in model_combinations:
    results[model] = dict()
    for dataset in dataset_combinations:
        results[model][dataset] = dict()
        for threshold in tresholds:
            results[model][dataset][threshold] = dict()
            for metric in metrics:
                results[model][dataset][threshold][metric] = None




In [4]:
with open('../all_datasets/predictive_models/catboost.pkl', 'rb') as f:
    model_all_datasets = pickle.load(f)

with open('../clickbait_dataset/predictive_models/catboost.pkl', 'rb') as f:
    model_clickbait_dataset_small = pickle.load(f)

with open('../clickbait_dataset_more_variables/predictive_models/lightgbm.pkl', 'rb') as f:
    model_clickbait_dataset_large = pickle.load(f)

with open('../fake_news/predictive_models/catboost.pkl', 'rb') as f:
    model_fake_news = pickle.load(f)

In [5]:
with open('../all_datasets/predictive_models/threshold.txt', 'rb') as f:
    threshold_all_datasets = float(f.read())

with open('../clickbait_dataset/predictive_models/threshold.txt', 'rb') as f:
    threshold_clickbait_dataset_small = float(f.read())

with open('../clickbait_dataset_more_variables/predictive_models/threshold.txt', 'rb') as f:
    threshold_clickbait_dataset_large = float(f.read())

with open('../fake_news/predictive_models/threshold.txt', 'rb') as f:
    threshold_fake_news = float(f.read())

In [6]:
with open('../all_datasets/predictive_models/scaler.pkl', 'rb') as f:
    scaler_all_datasets = pickle.load(f)

with open('../clickbait_dataset/predictive_models/scaler.pkl', 'rb') as f:
    scaler_clickbait_dataset_small = pickle.load(f)

with open('../clickbait_dataset_more_variables/predictive_models/scaler.pkl', 'rb') as f:
    scaler_clickbait_dataset_large = pickle.load(f)

with open('../fake_news/predictive_models/scaler.pkl', 'rb') as f:
    scaler_fake_news = pickle.load(f)


c:\Users\adamm\anaconda3\Lib\site-packages\sklearn\base.py:348: InconsistentVersionWarning: Trying to unpickle estimator StandardScaler from version 1.3.1 when using version 1.3.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


# All datasets data

In [ ]:
import numpy as np
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
for model in tqdm(model_combinations, desc='models'):
    if model == 'mod_all_datasets':
        scaler = cp.deepcopy(scaler_all_datasets)
        predictor = cp.deepcopy(model_all_datasets)
        opt_threshold = threshold_all_datasets
    elif model == 'mod_fake-news':
        scaler = cp.deepcopy(scaler_fake_news)
        predictor = cp.deepcopy(model_fake_news)
        opt_threshold = threshold_fake_news
    elif model == 'mod_clickbait-dataset-small':
        scaler = cp.deepcopy(scaler_clickbait_dataset_small)
        predictor = cp.deepcopy(model_clickbait_dataset_small)
        opt_threshold = threshold_clickbait_dataset_small
    elif model == 'mod_clickbait-dataset-large':
        scaler = cp.deepcopy(scaler_clickbait_dataset_large)
        predictor = cp.deepcopy(model_clickbait_dataset_large)
        opt_threshold = threshold_clickbait_dataset_large
       

    for dataset in tqdm(dataset_combinations, desc='datasets'):
        if dataset == 'data_all_datasets':
            df_temp = cp.deepcopy(df_all_datasets)
            dataset_name = 'all_datasets'
        elif dataset == 'data_fake-news':
            df_temp = cp.deepcopy(df_fake_news)
            dataset_name = 'fake-news'
        elif dataset == 'data_clickbait-dataset':
            df_temp = cp.deepcopy(df_clickbait_dataset)
            dataset_name = 'clickbait-dataset'

        for threhold, threshold_name in zip([0.5, opt_threshold], ['default', 'optimizedF1']):
            X = np.vstack(df_temp['title_vector_' + dataset_name].values)
            y_true = df_temp['is_clickbait']
            X_scaled = scaler.transform(X)

            y_pred_proba = predictor.predict_proba(X_scaled)[:,1]
            y_pred = (y_pred_proba > threhold).astype(int)


            results[model][dataset][threshold_name]['accuracy'] = accuracy_score(y_true, y_pred)
            results[model][dataset][threshold_name]['precision'] = precision_score(y_true, y_pred)
            results[model][dataset][threshold_name]['recall'] = recall_score(y_true, y_pred)
            results[model][dataset][threshold_name]['f1-score'] = f1_score(y_true, y_pred)
            results[model][dataset][threshold_name]['roc-auc'] = roc_auc_score(y_true, y_pred_proba)



    
